# callbacks.mixup

Type an introduction of the package here. Link to paper [mixup](https://arxiv.org/abs/1710.0941). 

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.callbacks.mixup import * 

In [ ]:
show_doc(MixUpCallback)

### <a id=MixUpCallback></a><em>class</em> `MixUpCallback`
(<code>learner</code>:[<code>Learner</code>](fastai.basic_train.html#Learner), <code>alpha</code>:<code>float</code>=`0.4`, <code>stack_x</code>:<code>bool</code>=`False`, <code>stack_y</code>:<code>bool</code>=`True`) -> <code>NoneType</code> :: Inherits ([<code>Callback</code>](fastai.callback.html#Callback))<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/callbacks/mixup.py#L6">[source]</a></div>


Callback that creates the mixed-up input and target

[`MixUpCallback`](/callbacks.mixup.html#MixUpCallback)

In [ ]:
show_doc(MixUpCallback.on_batch_begin)

#### <a id=on_batch_begin></a>`on_batch_begin`(<code>last_input</code>, <code>last_target</code>, <code>kwargs</code>)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/callbacks/mixup.py#L13">[source]</a></div>


Set HP before the step is done. Returns xb, yb (which can allow us to modify the input at that step if needed)

`MixUpCallback.on_batch_begin`

In [ ]:
show_doc(MixUpLoss)

### <a id=MixUpLoss></a><em>class</em> `MixUpLoss`(<code>crit</code>) :: Inherits ([<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/callbacks/mixup.py#L29">[source]</a></div>


Adapts the loss function to go with mixup.

[`MixUpLoss`](/callbacks.mixup.html#MixUpLoss)

In [ ]:
show_doc(MixUpLoss.forward)

#### <a id=forward></a>`forward`(<code>output</code>, <code>target</code>)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/callbacks/mixup.py#L36">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`MixUpLoss.forward`